# String Matching Debug Notebook

Use this notebook to debug why string matches occurred (or didn't occur) between LLM outputs and inputs.

In [4]:
from ao.runner.string_matching import (
    tokenize,
    is_content_match,
    compute_longest_match,
    MIN_MATCH_WORDS,
    MIN_COVERAGE_PRODUCT,
)

2026-01-14 16:25:38,598 - AO - INFO - DatabaseManager initialized with backend: local


## Input Strings

Paste the **output** text (from a previous LLM call) and the **input** text (from a new LLM call) below:

In [1]:
# Paste the OUTPUT text from a previous LLM call here
output_text = """
**Question**: Among the schools with the average score in Math over 560 in the SAT test, how many schools are directly charter-funded?

**1. Divide and Conquer:**

* **Main Question:** Among the schools with the average score in Math over 560 in the SAT test, how many schools are directly charter-funded?
    * **Analysis:** The question requires us to count the number of schools that meet two criteria: they have an average Math score over 560 and they are directly charter-funded. The relevant columns are `AvgScrMath` from the `satscores` table and `Charter Funding Type` from the `frpm` table.
    * **Pseudo SQL:** SELECT COUNT(*) FROM `satscores` AS `T1` INNER JOIN `frpm` AS `T2` ON `T1`.`cds` = `T2`.`CDSCode` WHERE <average score in Math over 560> AND <directly charter-funded>

    * **Sub-question 1:** average score in Math over 560
        * **Analysis:** This is a straightforward condition on the `AvgScrMath` column from the `satscores` table.
        * **Pseudo SQL:** `T1`.`AvgScrMath` > 560

    * **Sub-question 2:** directly charter-funded
        * **Analysis:** We need to filter the `frpm` table for schools that are directly charter-funded. This is indicated by the `Charter Funding Type` column.
        * **Pseudo SQL:** `T2`.`Charter Funding Type` = 'Directly funded'

**2. Assembling SQL:**

* **Sub-question 1 (average score in Math over 560):**
    * **SQL:** `T1`.`AvgScrMath` > 560

* **Sub-question 2 (directly charter-funded):**
    * **SQL:** `T2`.`Charter Funding Type` = 'Directly funded'

* **Main Question (count of schools):**
    * **SQL:** SELECT COUNT(*) FROM `satscores` AS `T1` INNER JOIN `frpm` AS `T2` ON `T1`.`cds` = `T2`.`CDSCode` WHERE `T1`.`AvgScrMath` > 560 AND `T2`.`Charter Funding Type` = 'Directly funded'

**3. Simplification and Optimization:**

* The SQL query from step 2 is already efficient. We have used an `INNER JOIN` to combine the relevant tables and applied the necessary filters in the `WHERE` clause.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT COUNT(*) FROM satscores AS T1 INNER JOIN frpm AS T2 ON T1.cds = T2.CDSCode WHERE T1.AvgScrMath > 560 AND T2.Charter Funding Type = 'Directly funded'
</FINAL_ANSWER>
"""

In [2]:
# Paste the INPUT text from the new LLM call here
input_text = """
You are an experienced database expert.
Now you need to generate a SQL query given the database information, a question and some additional information.
The database structure is defined by the following table schemas (comments after '--' provide additional column descriptions).
Note that the "Example Values" are actual values from the column. Some column might contain the values that are directly related to the question. Use it to help you justify which columns to use.

Given the table schema information description and the `Question`. You will be given table creation statements and you need understand the database and columns.

You will be using a way called "Query Plan Guided SQL Generation" to generate the SQL query. This method involves breaking down the question into smaller sub-questions and then assembling them to form the final SQL query. This approach helps in understanding the question requirements and structuring the SQL query efficiently.

Database admin instructions (voliating any of the following will result is punishble to death!):
1. **SELECT Clause:** 
    - Only select columns mentioned in the user's question. 
    - Avoid unnecessary columns or values.
2. **Aggregation (MAX/MIN):**
    - Always perform JOINs before using MAX() or MIN().
3. **ORDER BY with Distinct Values:**
    - Use `GROUP BY <column>` before `ORDER BY <column> ASC|DESC` to ensure distinct values.
4. **Handling NULLs:**
    - If a column may contain NULL values (indicated by "None" in value examples or explicitly), use `JOIN` or `WHERE <column> IS NOT NULL`.
5. **FROM/JOIN Clauses:**
    - Only include tables essential to answer the question.
6. **Strictly Follow Hints:**
    - Adhere to all provided hints.
7. **Thorough Question Analysis:**
    - Address all conditions mentioned in the question.
8. **DISTINCT Keyword:**
    - Use `SELECT DISTINCT` when the question requires unique values (e.g., IDs, URLs). 
    - Refer to column statistics ("Value Statics") to determine if `DISTINCT` is necessary.
9. **Column Selection:**
    - Carefully analyze column descriptions and hints to choose the correct column when similar columns exist across tables.
10. **String Concatenation:**
    - Never use `|| ' ' ||` or any other method to concatenate strings in the `SELECT` clause. 
11. **JOIN Preference:**
    - Prioritize `INNER JOIN` over nested `SELECT` statements.
12. **SQLite Functions Only:**
    - Use only functions available in SQLite.
13. **Date Processing:**
    - Utilize `STRFTIME()` for date manipulation (e.g., `STRFTIME('%Y', SOMETIME)` to extract the year).

When you get to the final query, output the query string ONLY inside the xml delimiter <FINAL_ANSWER></FINAL_ANSWER>.

Here are some examples

======= Example =======
**************************
【Table creation statements】
CREATE TABLE generalinfo
(
	id_restaurant INTEGER not null primary key,
	food_type TEXT null, -- examples: `thai`| `food type` description: the food type
	city TEXT null, -- description: the city where the restaurant is located in
);

CREATE TABLE location
(
	id_restaurant INTEGER not null primary key,
	street_name TEXT null, -- examples: `ave`, `san pablo ave`, `pablo ave`| `street name` description: the street name of the restaurant
	city TEXT null, -- description: the city where the restaurant is located in
	foreign key (id_restaurant) references generalinfo (id_restaurant) on update cascade on delete cascade,
);


**************************
【Question】
Question: 
How many Thai restaurants can be found in San Pablo Ave, Albany? 

Evidence:
Thai restaurant refers to food_type = 'thai'; San Pablo Ave Albany refers to street_name = 'san pablo ave' AND T1.city = 'albany'

**************************
【Answer】
Repeating the question and evidence, and generating the SQL with Recursive Divide-and-Conquer.
**Question**: How many Thai restaurants can be found in San Pablo Ave, Albany? 
**Evidence**: Thai restaurant refers to food_type = 'thai'; San Pablo Ave Albany refers to street_name = 'san pablo ave' AND T1.city = 'albany'

**Query Plan**:

** Preparation Steps:**
1. Initialize the process: Start preparing to execute the query.
2. Prepare storage: Set up storage space (registers) to hold temporary results, initializing them to NULL.
3. Open the location table: Open the location table so we can read from it.
4. Open the generalinfo table: Open the generalinfo table so we can read from it.

** Matching Restaurants:**
1. Start reading the location table: Move to the first row in the location table.
2. Check if the street matches: Look at the street_name column of the current row in location. If it’s not "san pablo ave," skip this row.
3. Identify the matching row: Store the identifier (row ID) of this location entry.
4. Find the corresponding row in generalinfo: Use the row ID from location to directly find the matching row in generalinfo.
5. Check if the food type matches: Look at the food_type column in generalinfo. If it’s not "thai," skip this row.
6. Check if the city matches: Look at the city column in generalinfo. If it’s not "albany," skip this row.

** Counting Restaurants:**
1. Prepare to count this match: If all checks pass, prepare to include this row in the final count.
2. Count this match: Increment the count for each row that meets all the criteria.
3. Move to the next row in location: Go back to the location table and move to the next row, repeating the process until all rows are checked.
4. Finalize the count: Once all rows have been checked, finalize the count of matching rows.
5. Prepare the result: Copy the final count to prepare it for output.

** Delivering the Result:**
1. Output the result: Output the final count, which is the number of restaurants that match all the specified criteria.
2. End the process: Stop the query execution process.
3. Setup phase: Before starting the actual query execution, the system prepares the specific values it will be looking for, like "san pablo ave," "thai," and "albany."

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT COUNT(T1.id_restaurant) FROM generalinfo AS T1 INNER JOIN location AS T2 ON T1.id_restaurant = T2.id_restaurant WHERE T1.food_type = 'thai' AND T1.city = 'albany' AND T2.street_name = 'san pablo ave'
</FINAL_ANSWER> 

===========
Example 1
**************************
【Database Info】
CREATE TABLE account (
    account_id INT PRIMARY KEY,
    district_id INT REFERENCES district(district_id),
    frequency VARCHAR(255) NOT NULL,
    date DATE NOT NULL
);
CREATE TABLE client (
    client_id INT PRIMARY KEY,
    gender CHAR(1) NOT NULL,
    birth_date DATE NOT NULL,
    district_id INT REFERENCES district(district_id)
);
CREATE TABLE district (
    district_id INT PRIMARY KEY,
    a4 VARCHAR(255) NOT NULL, -- Assuming A4 and A11 are strings due to examples
    a11 VARCHAR(255) NOT NULL
);
**************************
【Question】
Question: What is the gender of the youngest client who opened account in the lowest average salary branch?
Hint: Given that Later birthdate refers to younger age; A11 refers to average salary

**************************
【Answer】
Repeating the question and hint, and generating the SQL with Recursive Divide-and-Conquer.
**Question**: What is the gender of the youngest client who opened account in the lowest average salary branch?
**Hint**: Given that Later birthdate refers to younger age; A11 refers to average salary

**Query Plan**:

** Preparation Steps: **
1. Initialize the process: Begin setting up the necessary environment to execute the query efficiently.
2. Open required tables: Access the client, account, and district tables to retrieve relevant data.
3. Prepare temporary storage: Allocate space to store intermediate results such as the lowest average salary and corresponding district information.

** Identify the Branch with Lowest Average Salary: **
1. Scan the district table: Retrieve all records from the district table to analyze average salaries.
2. Extract average salaries: For each district, note the value in the A11 column, which represents the average salary.
3. Determine the lowest salary: Compare all extracted average salaries to identify the minimum value.
4. Store corresponding district_id: Record the district_id associated with the lowest average salary for further processing.

** Find Clients in the Identified District: **
1. Join client and account tables: Merge records where client.client_id matches account.account_id to associate clients with their accounts.
2. Filter by district_id: Select only those records where account.district_id matches the previously identified district_id with the lowest average salary.
3. Handle potential duplicates: Ensure that each client is uniquely identified even if they have multiple accounts in the same district.

** Identify the Youngest Client: **
1. Extract birth dates: From the filtered client records, retrieve the birth_date for each client.
2. Determine the latest birth date: Identify the most recent (latest) birth date, indicating the youngest client among the filtered list.
3. Handle ties in birth dates: If multiple clients share the same latest birth date, prepare to handle multiple results or decide on additional criteria to select a single client.

** Retrieve Gender Information: **
1. Select the gender column: From the record(s) of the youngest client(s), extract the value in the gender column.
2. Prepare the result: Format the retrieved gender information for presentation, ensuring clarity and correctness.

** Finalize and Deliver the Result: **
1. Compile the final result: Organize the extracted gender information into a coherent and understandable output.
2. Clean up resources: Close any open table connections and release temporary storage used during query execution.
3. Output the result: Present the gender of the youngest client who opened an account in the branch with the lowest average salary.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT `T1`.`gender`
  FROM `client` AS `T1`
  INNER JOIN `district` AS `T2`
  ON `T1`.`district_id` = `T2`.`district_id`
  ORDER BY `T2`.`A11` ASC, `T1`.`birth_date` DESC NULLS LAST
  LIMIT 1
</FINAL_ANSWER>

===========
Example 2 (dividing into two parallel sub-questions)
**************************
【Database Info】
CREATE TABLE games
(
	id INTEGER not null primary key,
	games_year INTEGER default NULL, -- `games year` description: the year of the game
);

CREATE TABLE games_city
(
	games_id INTEGER default NULL,
	city_id INTEGER default NULL, -- `city id` description: the id of the city that held the game Maps to city(id)
	foreign key (city_id) references city(id),
	foreign key (games_id) references games(id),
);

CREATE TABLE city
(
	id INTEGER not null primary key,
	city_name TEXT default NULL, -- examples: `London`
);

**************************
【Question】
Question:
From 1900 to 1992, how many games did London host?

Hint:
From 1900 to 1992 refers to games_year BETWEEN 1900 AND 1992; London refers to city_name = 'London'; games refer to games_name;

**************************
【Answer】

Repeating the question and hint, and generating the SQL with Recursive Divide-and-Conquer.
**Question**: From 1900 to 1992, how many games did London host?
**Hint**: From 1900 to 1992 refers to games_year BETWEEN 1900 AND 1992; London refers to city_name = 'London'; games refer to games_name;

**Query Plan**:

** Preparation Steps: **
1.Initialize the process: Set up the environment to begin query execution, including necessary variables and temporary storage.
2. Open required tables: Open the games_city, city, and games tables to access relevant data.
3. Prepare filtering values: Set up the specific values to filter the data, such as the year range (1900-1992) and the city name 'London'.

** Filter and Identify Relevant Data: **
1. Scan games_city table: Retrieve records from the games_city table to match games with the cities where they were hosted.
2. Fetch the corresponding city_id: For each row in games_city, extract the city_id to find out which city hosted the game.
3 .Match city_id with city_name: Use the city_id to look up the corresponding city_name in the city table.
4. Filter by city_name = 'London': Select only those rows where the city_name is 'London'.

** Further Filter by Year Range: **
1. Extract games_id: For rows that match 'London', retrieve the games_id from the games_city table.
2. Find matching games_year: Use the games_id to look up the corresponding games_year in the games table.
3. Filter by games_year between 1900 and 1992: Select only those rows where the games_year falls within the specified range (1900-1992).

** Count the Matching Rows: **
1. Initialize the count: Prepare to count the number of matching rows that meet all the criteria.
2. Count the valid entries: For each row that matches the conditions (city_name = 'London' and games_year between 1900 and 1992), increment the count.
3. Store the final count: Once all rows have been processed, store the total count as the final result.

** Finalize and Deliver the Result: **
1. Prepare the result for output: Format the final count of games hosted by London between 1900 and 1992.
2. Output the final count: Deliver the count as the query result.
3. Clean up resources: Close any open table connections and release temporary storage used during query execution.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT COUNT(T3.id) FROM games_city AS T1 INNER JOIN city AS T2 ON T1.city_id = T2.id INNER JOIN games AS T3 ON T1.games_id = T3.id WHERE T2.city_name = 'London' AND T3.games_year BETWEEN 1900 AND 1992
</FINAL_ANSWER> 

=============

Example 3
**************************
【Table creation statements】
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    department_id INT,
    salary INT 
);
**************************
【Question】
Question: How many employees earn over $100,000?

**************************
【Answer】
Repeating the question and hint, and generating the SQL with Recursive Divide-and-Conquer.
**Question:** How many employees earn over $100,000?

** Query Plan**:

** Preparation Steps: **
1.cInitialize the process: Begin by setting up the environment for query execution, including initializing variables and temporary storage.
2. Open the employees table: Access the employees table to retrieve the relevant data.

** Filtering Employees by Salary: **
1. Scan the employees table: Begin reading rows from the employees table.
2. Fetch the salary column: For each row, retrieve the value from the salary column.
3. Compare salary against $100,000: Check if the salary value is greater than $100,000.
4. Identify matching rows: For rows where the salary exceeds $100,000, prepare to count these entries.

** Counting the Matches: **
1. Initialize the count: Set up a counter to keep track of how many employees meet the salary condition.
2. Increment the count: For each row where the salary is above $100,000, increment the counter.
3. Store the final count: Once all rows have been processed, store the total count of matching employees.

** Finalize and Deliver the Result: **
1. Prepare the result for output: Format the final count for presentation.
2. Output the final count: Deliver the count as the query result, indicating how many employees earn over $100,000.
3. Clean up resources: Close the employees table and release any temporary storage used during query execution.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT COUNT(*) FROM employees WHERE salary > 100000;
</FINAL_ANSWER>

====== Example 5 =======
**************************
【Table creation statements】
CREATE TABLE Airlines
(
	FL_DATE TEXT, -- examples: `2018/8/9`| `flight date` description: flight date
	ORIGIN TEXT, -- examples: `SAN`| description: airport of origin
	DEST TEXT, -- examples: `SAN`| `destination` description: Destination airport
	FOREIGN KEY (ORIGIN) REFERENCES Airports(Code),
	FOREIGN KEY (DEST) REFERENCES Airports(Code),
);

CREATE TABLE Airports
(
	Code TEXT primary key,
	Description TEXT,
);


**************************
【Question】
Question: 
How many flights were there from San Diego International airport to Los Angeles International airport in the August of 2018? 

Evidence:
flights from refers to ORIGIN; San Diego International airport refers to Description = 'San Diego, CA: San Diego International'; flights to refers to DEST; Los Angeles International airport refers to Description = 'Los Angeles, CA: Los Angeles International'; in the August of 2018 refers to FL_DATE like '2018/8%';


**************************
【Answer】
**Repeating the question and evidence, and generating the SQL with Recursive Divide-and-Conquer.**

**Question**: How many flights were there from San Diego International airport to Los Angeles International airport in the August of 2018? 
**Evidence**: flights from refers to ORIGIN; San Diego International airport refers to Description = 'San Diego, CA: San Diego International'; flights to refers to DEST; Los Angeles International airport refers to Description = 'Los Angeles, CA: Los Angeles International'; in the August of 2018 refers to FL_DATE like '2018/8%';

** Query Plan**:

** Preparation Steps: **
1. Initialize the process: Set up the environment and prepare for query execution by initializing variables and temporary storage.
2. Open required tables: Open the Airlines and Airports tables to access relevant data.

** Identify Relevant Flights: **
1. Fetch the FL_DATE column: Start reading the FL_DATE column from the Airlines table.
2. Filter by August 2018: Use the condition FL_DATE LIKE '2018/8%' to filter flights that occurred in August 2018.
3. Join with Airports for ORIGIN: Identify flights originating from 'San Diego, CA: San Diego International' by joining the Airlines table with the Airports table on the ORIGIN field.
4. Join with Airports for DEST: Similarly, identify flights destined for 'Los Angeles, CA: Los Angeles International' by joining the Airlines table with the Airports table on the DEST field.

** Count the Matching Flights: **
1. Initialize the count: Set up a counter to keep track of how many flights match the criteria.
2. Increment the count: For each flight that meets the conditions (originating from San Diego International and destined for Los Angeles International in August 2018), increment the counter.
3. Store the final count: Once all rows have been processed, store the total count of matching flights.

** Finalize and Deliver the Result: **
1. Prepare the result for output: Format the final count for presentation, ensuring clarity and correctness.
2. Output the final count: Deliver the count as the query result, indicating how many flights met the specified criteria.
3. Clean up resources: Close any open table connections and release temporary storage used during query execution.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT COUNT(FL_DATE) FROM Airlines WHERE FL_DATE LIKE '2018/8%' AND ORIGIN = ( SELECT T2.ORIGIN FROM Airports AS T1 INNER JOIN Airlines AS T2 ON T1.Code = T2.ORIGIN WHERE T1.Description = 'San Diego, CA: San Diego International' ) AND DEST = ( SELECT T4.DEST FROM Airports AS T3 INNER JOIN Airlines AS T4 ON T3.Code = T4.DEST WHERE T3.Description = 'Los Angeles, CA: Los Angeles International' )
</FINAL_ANSWER> 

===== Example 5 ========

**************************
【Table creation statements】
CREATE TABLE businesses
(
        `business_id` INTEGER NOT NULL,
        `name` TEXT NOT NULL, -- description: the name of the eatery
        PRIMARY KEY (`business_id`),
);

CREATE TABLE inspections
(
        `business_id` INTEGER NOT NULL, -- `business id` description: the unique id of the business
        `score` INTEGER DEFAULT NULL, -- description: the inspection score
        `date` DATE NOT NULL, -- examples: `2014-01-24`
        FOREIGN KEY (`business_id`) REFERENCES `businesses` (`business_id`),
);

CREATE TABLE violations
(
        `business_id` INTEGER NOT NULL,
        `date` DATE NOT NULL, -- examples: `2016-05-03`
        FOREIGN KEY (`business_id`) REFERENCES `businesses` (`business_id`),
);


**************************
【Question】
Question: 
What are the names of the establishments that met all the required standards for 4 consecutive years? 

Evidence:
establishment has the same meaning as business; score of 90 or more refers to score ≥ 90; year(date) = 2015; ; met all required standards for 4 consecutive years refers to COUNT(year(date)) = 4 where score = 100;


**************************
【Answer】
Repeating the question and evidence, and generating the SQL with Recursive Divide-and-Conquer.

**Question**: What are the names of the establishments that met all the required standards for 4 consecutive years? 
**Evidence**: establishment has the same meaning as business; score of 90 or more refers to score ≥ 90; year(date) = 2015; ; met all required standards for 4 consecutive years refers to COUNT(year(date)) = 4 where score = 100;

** Query Plan**:

** Preparation Steps: **
1. Initialize the process: Set up the environment and prepare for query execution, including initializing variables and temporary storage.
2. Open required tables: Open the businesses, inspections, and violations tables to access relevant data.

** Filter and Identify Relevant Inspections: **
1. Scan the inspections table: Start reading rows from the inspections table.
2. Filter by score of 100: Select only those inspections where the score is 100, indicating that the establishment met all required standards.
3. Extract year from the inspection date: Use the STRFTIME('%Y', date) function to extract the year from the inspection date.
4. Join with businesses table: Match each inspection to the corresponding business by joining on business_id.

** Identify Businesses Meeting Standards for 4 Consecutive Years: **
1. Aggregate by business and year: Group the data by business name and the extracted year to count the number of years each business met the required standards.
3. Apply row numbering: Use ROW_NUMBER() with a partition by business name and order by year to identify consecutive years.
3. Filter for 4 consecutive years: Group by business name and ensure that the count of years with the required score is exactly 4, indicating 4 consecutive years of meeting the standards.

** Count and Finalize the Results: **
1. Count the matching businesses: For each business, count the number of years that meet the criteria.
2. Select distinct business names: Extract the names of businesses that have met the required standards for 4 consecutive years.
3. Store and prepare the result: Once all businesses have been processed, store the result and prepare it for output.

** Deliver the Final Result: **
1. Prepare the result for output: Format the final list of business names for presentation.
2. Output the final result: Deliver the names of the businesses that met the required standards for 4 consecutive years.
3. Clean up resources: Close any open table connections and release temporary storage used during query execution.

**Final Optimized SQL Query:**

<FINAL_ANSWER>
SELECT DISTINCT T4.name FROM ( SELECT T3.name, T3.years, row_number() OVER (PARTITION BY T3.name ORDER BY T3.years) AS rowNumber FROM ( SELECT DISTINCT name, STRFTIME('%Y', `date`) AS years FROM inspections AS T1 INNER JOIN businesses AS T2 ON T1.business_id = T2.business_id WHERE T1.score = 100 ) AS T3 ) AS T4 GROUP BY T4.name, date(T4.years || '-01-01', '-' || (T4.rowNumber - 1) || ' years') HAVING COUNT(T4.years) = 4
</FINAL_ANSWER>
===========

Now is the real question, following the instruction and examples, You should:
1. Generate a QUERY PLAN for the given question and evidence.
2. Return the final optimized SQL query.

**************************
【Table creation statements】
CREATE TABLE schools
(
	Longitude REAL null, -- Example Values: `(-122.09713,)`, `(-121.99391,)`, `(-122.26815,)` | Value Statics: Total count 12863 - Distinct count 11278 - Null count 4823   
	District TEXT not null, -- Example Values: `('Alameda County Office of Education',)`, `('California School for the Blind (State Special Schl)',)`, `('California School for the Deaf-Fremont (State Special Schl)',)` | Value Statics: Total count 17686 - Distinct count 1411 - Null count 0   
	NCESSchool TEXT null, -- Example Values: `('10546',)`, `('10947',)`, `('12283',)` | Value Statics: Total count 12646 - Distinct count 12321 - Null count 5040   
	StatusType TEXT not null, -- Example Values: `Active`, `Closed`, `Merged`, `Pending` | Value Statics: Total count 17686 - Distinct count 4 - Null count 0   
	AdmFName3 TEXT null, -- Example Values: `('Drew',)`, `('Irma',)`, `('Vickie',)` | Value Statics: Total count 42 - Distinct count 40 - Null count 17644   
	ClosedDate DATE null, -- Example Values: `('2015-07-31',)`, `('2015-06-30',)`, `('1989-06-30',)` | Value Statics: Total count 5694 - Distinct count 899 - Null count 11992   
	MailCity TEXT null, -- Example Values: `('Hayward',)`, `('Newark',)`, `('Oakland',)` | Value Statics: Total count 17394 - Distinct count 1132 - Null count 292   
	City TEXT null, -- Example Values: `('Hayward',)`, `('Newark',)`, `('Oakland',)` | Value Statics: Total count 17393 - Distinct count 1165 - Null count 293   
	Magnet INTEGER null, -- Example Values: `0`, `1` | Value Statics: Total count 10610 - Distinct count 2 - Null count 7076   
	CharterNum TEXT null, -- Example Values: `('0728',)`, `('0811',)`, `('1049',)` | Value Statics: Total count 1801 - Distinct count 1763 - Null count 15885  | Column Description: The charter school number, | Value Description: 4-digit number assigned to a charter school.
	School TEXT null, -- Example Values: `MethodSchools` | Value Statics: Total count 16317 - Distinct count 13875 - Null count 1369  | Column Description: School 
	MailStrAbr TEXT null, -- Example Values: `('313 West Winton Ave.',)`, `('39899 Balentine Dr., Ste. 335',)`, `('1515 Webster St.',)` | Value Statics: Total count 17394 - Distinct count 12429 - Null count 292   
	Charter INTEGER null, -- Example Values: `1`, `0` | Value Statics: Total count 16317 - Distinct count 2 - Null count 1369   
	AdmFName1 TEXT null, -- Example Values: `('L Karen',)`, `('Laura',)`, `('Clifford',)` | Value Statics: Total count 11700 - Distinct count 2327 - Null count 5986   
	State TEXT null, -- Example Values: `CA` | Value Statics: Total count 17393 - Distinct count 1 - Null count 293   
	MailState TEXT null, -- Example Values: `CA` | Value Statics: Total count 17394 - Distinct count 1 - Null count 292   
	AdmFName2 TEXT null, -- Example Values: `('Sau-Lim (Lance)',)`, `('Jennifer',)`, `('Annalisa',)` | Value Statics: Total count 431 - Distinct count 285 - Null count 17255   | Value Description: SAME as 1
	Zip TEXT null, -- Example Values: `('94544-1136',)`, `('94560-5359',)`, `('94612-3355',)` | Value Statics: Total count 17393 - Distinct count 11184 - Null count 293   
	AdmEmail2 TEXT null, -- Example Values: `('stsang@unityhigh.org',)`, `('jkoelling@efcps.net',)`, `('annalisa.moore@neaclc.org',)` | Value Statics: Total count 424 - Distinct count 382 - Null count 17262   
	Latitude REAL null, -- Example Values: `(37.658212,)`, `(37.521436,)`, `(37.80452,)` | Value Statics: Total count 12863 - Distinct count 11436 - Null count 4823   
	FundingType TEXT null, -- Example Values: `Directly funded`, `Locally funded`, `Not in CS funding model` | Value Statics: Total count 1642 - Distinct count 3 - Null count 16044  | Column Description: Indicates the charter school funding type | Value Description: Values are as follows:  ·       Not in CS (California School) funding model  ·       Locally funded  ·       Directly funded
	SOCType TEXT null, -- Example Values: `('K-12 Schools (Public)',)`, `('High Schools (Public)',)`, `('Elementary Schools (Public)',)` | Value Statics: Total count 16317 - Distinct count 20 - Null count 1369   
	MailStreet TEXT null, -- Example Values: `('313 West Winton Avenue',)`, `('39899 Balentine Drive, Suite 335',)`, `('1515 Webster Street',)` | Value Statics: Total count 17394 - Distinct count 12395 - Null count 292   
	Phone TEXT null, -- Example Values: `('(510) 887-0152',)`, `('(510) 596-8901',)`, `('(510) 686-4131',)` | Value Statics: Total count 11717 - Distinct count 10632 - Null count 5969  | Column Description: Phone 
	EdOpsCode TEXT null, -- Example Values: `TRAD`, `JUV`, `COMM`, `OPP`, `SPEC` | Value Statics: Total count 11975 - Distinct count 13 - Null count 5711   
	Virtual TEXT null, -- Example Values: `P`, `N`, `F` | Value Statics: Total count 10818 - Distinct count 3 - Null count 6868   
	DOCType TEXT not null, -- Example Values: `County Office of Education (COE)`, `State Special Schools`, `Non-School Locations`, `Joint Powers Authority (JPA)`, `Unified School District` | Value Statics: Total count 17686 - Distinct count 12 - Null count 0   
	County TEXT not null, -- Example Values: `('Alameda',)`, `('Alpine',)`, `('Amador',)` | Value Statics: Total count 17686 - Distinct count 58 - Null count 0   
	GSoffered TEXT null, -- Example Values: `('K-12',)`, `('9-12',)`, `('K-8',)` | Value Statics: Total count 13804 - Distinct count 94 - Null count 3882   
	AdmEmail3 TEXT null, -- Example Values: `('dsarratore@vincentacademy.org',)`, `('gmunoz@piedmont.k12.ca.us',)`, `('vickiechang@acoe.org',)` | Value Statics: Total count 42 - Distinct count 42 - Null count 17644   
	GSserved TEXT null, -- Example Values: `('K-12',)`, `('9-12',)`, `('K-7',)` | Value Statics: Total count 11943 - Distinct count 81 - Null count 5743   
	SOC TEXT null, -- Example Values: `('65',)`, `('66',)`, `('60',)` | Value Statics: Total count 16317 - Distinct count 20 - Null count 1369   
	AdmLName1 TEXT null, -- Example Values: `('Monroe',)`, `('Robell',)`, `('Thompson',)` | Value Statics: Total count 11700 - Distinct count 6394 - Null count 5986   
	EILCode TEXT null, -- Example Values: `ELEMHIGH`, `HS`, `ELEM`, `UG`, `INTMIDJR` | Value Statics: Total count 16317 - Distinct count 7 - Null count 1369   
	OpenDate DATE null, -- Example Values: `('2005-08-29',)`, `('2006-08-28',)`, `('2008-08-21',)` | Value Statics: Total count 16317 - Distinct count 1406 - Null count 1369   
	AdmLName3 TEXT null, -- Example Values: `('Sarratore',)`, `('Munoz',)`, `('Chang',)` | Value Statics: Total count 42 - Distinct count 42 - Null count 17644   
	Ext TEXT null, -- Example Values: `('130',)`, `('1240',)`, `('1200',)` | Value Statics: Total count 540 - Distinct count 379 - Null count 17146   
	AdmLName2 TEXT null, -- Example Values: `('Tsang',)`, `('Koelling',)`, `('Moore',)` | Value Statics: Total count 431 - Distinct count 363 - Null count 17255   
	Website TEXT null, -- Example Values: `('www.acoe.org',)`, `('www.envisionacademy.org/',)`, `('www.aspirepublicschools.org',)` | Value Statics: Total count 6964 - Distinct count 4082 - Null count 10722   
	NCESDist TEXT null, -- Example Values: `('0691051',)`, `('0600002',)`, `('0600003',)` | Value Statics: Total count 16656 - Distinct count 1193 - Null count 1030   
	MailZip TEXT null, -- Example Values: `('94544-1136',)`, `('94560-5359',)`, `('94612',)` | Value Statics: Total count 17394 - Distinct count 10298 - Null count 292   
	CDSCode TEXT not null primary key,
	Street TEXT null, -- Example Values: `('313 West Winton Avenue',)`, `('39899 Balentine Drive, Suite 335',)`, `('1515 Webster Street',)` | Value Statics: Total count 17392 - Distinct count 13593 - Null count 294   
	StreetAbr TEXT null, -- Example Values: `('313 West Winton Ave.',)`, `('39899 Balentine Dr., Ste. 335',)`, `('1515 Webster St.',)` | Value Statics: Total count 17392 - Distinct count 13633 - Null count 294   
	EILName TEXT null, -- Example Values: `Elementary-High Combination`, `High School`, `Elementary`, `Ungraded`, `Intermediate/Middle/Junior High` | Value Statics: Total count 16317 - Distinct count 7 - Null count 1369   
	LastUpdate DATE not null, -- Example Values: `('2015-06-23',)`, `('2015-09-01',)`, `('2015-06-18',)` | Value Statics: Total count 17686 - Distinct count 757 - Null count 0   
	AdmEmail1 TEXT null, -- Example Values: `('lkmonroe@acoe.org',)`, `('laura@envisionacademy.org',)`, `('cliffordt@communityschoolforcreativeeducation.org',)` | Value Statics: Total count 11674 - Distinct count 10492 - Null count 6012   
	EdOpsName TEXT null, -- Example Values: `Traditional`, `Juvenile Court School`, `County Community School`, `Opportunity School`, `Special Education School` | Value Statics: Total count 11975 - Distinct count 13 - Null count 5711   
	DOC TEXT not null, -- Example Values: `00`, `31`, `34`, `42`, `54` | Value Statics: Total count 17686 - Distinct count 12 - Null count 0   
);

CREATE TABLE frpm
(
	`Educational Option Type` TEXT null, -- Example Values: `Traditional`, `Juvenile Court School`, `County Community School`, `State Special School`, `Alternative School of Choice` | Value Statics: Total count 9941 - Distinct count 12 - Null count 45   
	`Percent (%) Eligible Free (Ages 5-17)` REAL null, -- Example Values: `(0.516822429906542,)`, `(0.484042553191489,)`, `(0.556521739130435,)` | Value Statics: Total count 9908 - Distinct count 8552 - Null count 78   
	`Charter School Number` TEXT null, -- Example Values: `('0728',)`, `('0811',)`, `('1049',)` | Value Statics: Total count 1167 - Distinct count 1152 - Null count 8819  | Column Description: Charter School Number 
	`2013-14 CALPADS Fall 1 Certification Status` INTEGER null, -- Example Values: `1` | Value Statics: Total count 9986 - Distinct count 1 - Null count 0   
	`FRPM Count (Ages 5-17)` REAL null, -- Example Values: `(702.0,)`, `(182.0,)`, `(168.0,)` | Value Statics: Total count 9914 - Distinct count 1330 - Null count 72   
	`Enrollment (Ages 5-17)` REAL null, -- Example Values: `(1070.0,)`, `(376.0,)`, `(230.0,)` | Value Statics: Total count 9972 - Distinct count 1845 - Null count 14   
	`NSLP Provision Status` TEXT null, -- Example Values: `Breakfast Provision 2`, `Provision 2`, `CEP`, `Lunch Provision 2`, `Provision 1` | Value Statics: Total count 1847 - Distinct count 7 - Null count 8139   
	`School Name` TEXT null, -- Example Values: `MethodSchools` | Value Statics: Total count 9986 - Distinct count 8652 - Null count 0  | Column Description: School Name 
	`Free Meal Count (K-12)` REAL null, -- Example Values: `(565.0,)`, `(186.0,)`, `(134.0,)` | Value Statics: Total count 9930 - Distinct count 1216 - Null count 56   
	IRC INTEGER null, -- Example Values: `1`, `0` | Value Statics: Total count 9941 - Distinct count 2 - Null count 45   
	foreign key (CDSCode) references schools (CDSCode),
	`High Grade` TEXT null, -- Example Values: `12`, `8`, `5`, `Adult`, `6` | Value Statics: Total count 9986 - Distinct count 17 - Null count 0   
	`School Code` TEXT null, -- Example Values: `('0109835',)`, `('0112607',)`, `('0118489',)` | Value Statics: Total count 9986 - Distinct count 9942 - Null count 0   
	`District Code` INTEGER null, -- Example Values: `(10017,)`, `(31609,)`, `(31617,)` | Value Statics: Total count 9986 - Distinct count 1012 - Null count 0   
	`Percent (%) Eligible FRPM (Ages 5-17)` REAL null, -- Example Values: `(0.65607476635514,)`, `(0.484042553191489,)`, `(0.730434782608696,)` | Value Statics: Total count 9914 - Distinct count 8557 - Null count 72   
	`Charter School (Y/N)` INTEGER null, -- Example Values: `1`, `0` | Value Statics: Total count 9941 - Distinct count 2 - Null count 45  | Column Description: Charter School (Y/N) | Value Description: 0: N; 1: Y
	`District Name` TEXT null, -- Example Values: `('Alameda County Office of Education',)`, `('California School for the Blind (State Special Schl)',)`, `('California School for the Deaf-Fremont (State Special Schl)',)` | Value Statics: Total count 9986 - Distinct count 1000 - Null count 0   
	`County Name` TEXT null, -- Example Values: `('Alameda',)`, `('Alpine',)`, `('Amador',)` | Value Statics: Total count 9986 - Distinct count 58 - Null count 0   
	`FRPM Count (K-12)` REAL null, -- Example Values: `(715.0,)`, `(186.0,)`, `(175.0,)` | Value Statics: Total count 9936 - Distinct count 1361 - Null count 50   
	`School Type` TEXT null, -- Example Values: `K-12 Schools (Public)`, `High Schools (Public)`, `Elementary Schools (Public)`, `Juvenile Court Schools`, `County Community` | Value Statics: Total count 9941 - Distinct count 17 - Null count 45  | Column Description: School Type 
	`Percent (%) Eligible Free (K-12)` REAL null, -- Example Values: `(0.519779208831647,)`, `(0.470886075949367,)`, `(0.549180327868853,)` | Value Statics: Total count 9930 - Distinct count 8658 - Null count 56   
	`County Code` TEXT null, -- Example Values: `('01',)`, `('02',)`, `('03',)` | Value Statics: Total count 9986 - Distinct count 58 - Null count 0   
	`Low Grade` TEXT null, -- Example Values: `K`, `9`, `1`, `P`, `6` | Value Statics: Total count 9986 - Distinct count 15 - Null count 0   
	`Percent (%) Eligible FRPM (K-12)` REAL null, -- Example Values: `(0.657773689052438,)`, `(0.470886075949367,)`, `(0.717213114754098,)` | Value Statics: Total count 9936 - Distinct count 8623 - Null count 50   
	`Enrollment (K-12)` REAL null, -- Example Values: `(1087.0,)`, `(395.0,)`, `(244.0,)` | Value Statics: Total count 9986 - Distinct count 1882 - Null count 0   
	`District Type` TEXT null, -- Example Values: `County Office of Education (COE)`, `State Special Schools`, `Unified School District`, `Elementary School District`, `High School District` | Value Statics: Total count 9986 - Distinct count 8 - Null count 0   
	`Free Meal Count (Ages 5-17)` REAL null, -- Example Values: `(553.0,)`, `(182.0,)`, `(128.0,)` | Value Statics: Total count 9908 - Distinct count 1205 - Null count 78   
	CDSCode TEXT not null primary key,
	`Charter Funding Type` TEXT null, -- Example Values: `Directly funded`, `Locally funded`, `Not in CS funding model` | Value Statics: Total count 1167 - Distinct count 3 - Null count 8819  | Column Description: Charter Funding Type 
	`Academic Year` TEXT null, -- Example Values: `2014-2015` | Value Statics: Total count 9986 - Distinct count 1 - Null count 0   
);

CREATE TABLE satscores
(
	AvgScrWrite INTEGER null, -- Example Values: `(417,)`, `(505,)`, `(395,)` | Value Statics: Total count 1673 - Distinct count 267 - Null count 596   
	sname TEXT null, -- Example Values: `('FAME Public Charter',)`, `('Envision Academy for Arts & Technology',)`, `('Aspire California College Preparatory Academy',)` | Value Statics: Total count 1749 - Distinct count 1665 - Null count 520 | Column Name Meaning: school name | Column Description: school name 
	AvgScrRead INTEGER null, -- Example Values: `(418,)`, `(503,)`, `(397,)` | Value Statics: Total count 1673 - Distinct count 269 - Null count 596 | Column Name Meaning: average scores in Reading | Column Description: average scores in Reading | Value Description: average scores in Reading
	enroll12 INTEGER not null, -- Example Values: `(398,)`, `(62,)`, `(75,)` | Value Statics: Total count 2269 - Distinct count 835 - Null count 0   
	cname TEXT null, -- Example Values: `('Alameda',)`, `('Amador',)`, `('Butte',)` | Value Statics: Total count 2269 - Distinct count 57 - Null count 0   
	NumGE1500 INTEGER null, -- Example Values: `(14,)`, `(9,)`, `(5,)` | Value Statics: Total count 1673 - Distinct count 368 - Null count 596 | Column Name Meaning: Number of Test Takers Whose Total SAT Scores Are Greater or Equal to 1500 | Column Description: Number of Test Takers Whose Total SAT Scores Are Greater or Equal to 1500 | Value Description: Number of Test Takers Whose Total SAT Scores Are Greater or Equal to 1500    Excellence Rate = NumGE1500 / NumTstTakr
	AvgScrMath INTEGER null, -- Example Values: `(418,)`, `(546,)`, `(387,)` | Value Statics: Total count 1673 - Distinct count 295 - Null count 596 | Column Name Meaning: average scores in Math | Column Description: average scores in Math | Value Description: average scores in Math
	cds TEXT not null primary key,
	NumTstTakr INTEGER not null, -- Example Values: `(88,)`, `(17,)`, `(71,)` | Value Statics: Total count 2269 - Distinct count 547 - Null count 0 | Column Name Meaning: Number of Test Takers | Column Description: Number of Test Takers in this school | Value Description: number of test takers in each school
	dname TEXT null, -- Example Values: `('Alameda County Office of Education',)`, `('Alameda Unified',)`, `('Albany City Unified',)` | Value Statics: Total count 2269 - Distinct count 520 - Null count 0   
	foreign key (cds) references schools (CDSCode),
	rtype TEXT not null, -- Example Values: `D`, `S` | Value Statics: Total count 2269 - Distinct count 2 - Null count 0   
);

**************************
【Question】
Question: 
Among the schools with the average score in Math over 560 in the SAT test, how many schools are directly charter-funded?

Evidence:


**************************
【Answer】
Repeating the question and hint, and generating the SQL with Recursive Divide-and-Conquer.
"""

## Run Matching Analysis

In [5]:
# Tokenize both strings
output_words = tokenize(output_text)
input_words = tokenize(input_text)

print(f"Output: {len(output_words)} words")
print(f"Input: {len(input_words)} words")
print()

# Compute longest match
match_len = compute_longest_match(output_words, input_words)
print(f"Longest contiguous match: {match_len} words")
print()

# Check if it's a match
is_match, match_type, match_len, coverage = is_content_match(output_words, input_words)

print("=" * 50)
print(f"MATCH: {is_match}")
if is_match:
    print(f"Match type: {match_type}")
print(f"Match length: {match_len} words")
print(f"Coverage product: {coverage:.4f}")
print("=" * 50)
print()

# Show thresholds
print(f"Thresholds:")
print(f"  MIN_MATCH_WORDS: {MIN_MATCH_WORDS}")
print(f"  MIN_COVERAGE_PRODUCT: {MIN_COVERAGE_PRODUCT}")
print()

# Explain why it matched or didn't
if match_len >= MIN_MATCH_WORDS:
    print(f"Matched via absolute threshold: {match_len} >= {MIN_MATCH_WORDS}")
elif len(output_words) > 0 and len(input_words) > 0:
    output_coverage = match_len / len(output_words)
    input_coverage = match_len / len(input_words)
    coverage_product = output_coverage * input_coverage
    print(f"Output coverage: {output_coverage:.4f} ({match_len}/{len(output_words)})")
    print(f"Input coverage: {input_coverage:.4f} ({match_len}/{len(input_words)})")
    print(f"Coverage product: {coverage_product:.4f}")
    if coverage_product >= MIN_COVERAGE_PRODUCT:
        print(f"Matched via coverage: {coverage_product:.4f} >= {MIN_COVERAGE_PRODUCT}")
    else:
        print(f"No match: coverage {coverage_product:.4f} < {MIN_COVERAGE_PRODUCT}")

Output: 285 words
Input: 5495 words

Longest contiguous match: 22 words

MATCH: True
Match type: absolute
Match length: 22 words
Coverage product: 0.0003

Thresholds:
  MIN_MATCH_WORDS: 10
  MIN_COVERAGE_PRODUCT: 0.1

Matched via absolute threshold: 22 >= 10


## Inspect Tokenized Words

In [ ]:
# Show first 50 words of each
print("Output words (first 50):")
print(output_words[:50])
print()
print("Input words (first 50):")
print(input_words[:50])

## Find the Matching Substring

In [6]:
from difflib import SequenceMatcher

if output_words and input_words:
    sm = SequenceMatcher(None, output_words, input_words, autojunk=False)
    match = sm.find_longest_match(0, len(output_words), 0, len(input_words))
    
    print(f"Match location:")
    print(f"  Output index: {match.a} to {match.a + match.size}")
    print(f"  Input index: {match.b} to {match.b + match.size}")
    print(f"  Length: {match.size} words")
    print()
    
    if match.size > 0:
        matched_words = output_words[match.a:match.a + match.size]
        print(f"Matched text ({match.size} words):")
        print(" ".join(matched_words))
else:
    print("One or both strings are empty after tokenization.")

Match location:
  Output index: 0 to 22
  Input index: 5459 to 5481
  Length: 22 words

Matched text (22 words):
question among the schools with the average score in math over 560 in the sat test how many schools are directly charterfunded
